In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model
import matplotlib.pyplot as plt



df = pd.read_excel(r'Copy of Extraction PT4 14x59 din 2016 pana in prezent - analysis S1928 (003) (3).xlsx', sheet_name='TL4 -40 Nm' )
dataset = df.values
X = dataset[1:,23:43]
Y = dataset[1:,14]
X = X.astype('float')
Y = Y.astype('float')

##0,12,4,16
d1 = pd.DataFrame(X)
d2 = pd.DataFrame(Y)
d1.drop([1,2,5,6,7,8,9,10,11,13,14,17,18,19], axis=1, inplace=True)
d2.replace(0.  , np.nan, inplace=True)
d2.replace(-1. , np.nan, inplace=True)
d2.replace(1.  , np.nan, inplace=True)
d2.replace(2.  , np.nan, inplace=True)
d2.replace(3.  , np.nan, inplace=True)
d2.replace(4.  , np.nan, inplace=True)
d2.replace(5.  , np.nan, inplace=True)
d2.replace(6.  , np.nan, inplace=True)


d1[5] = d2[0]
d1.dropna(inplace=True)
# d1.isnull().any() # Verifica daca avem vre-o valoare de tip NaN

X = d1.values[0:,:-1]
Y = d1.values[0:, -1:]


min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
Y_scaler = preprocessing.MinMaxScaler()
Y_scale = Y_scaler.fit_transform(Y)




X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape




def tree_model(x, dropout=0.2):
    model = Sequential()
    model.add(Dense(x, input_dim=6, kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/4),  kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def basic_model_3(x):
     # create model
    model = Sequential()
    model.add(Dense(x, input_dim=6, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(x, input_dim=4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(x, input_dim=2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def etree_model(x, dropout=0.2):
    model = Sequential()
    model.add(Dense(x, input_dim=6, kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x*2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/4),  kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


def err(model, x):
    emax = 1.
    emin = 1.
    for i in range(x):
        p = model.predict(X_train[i].reshape(1, 6))
        #print(p, Y_train[i], p/float(Y_train[i]))
        pp = Y_scaler.inverse_transform(p)
        py = Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
        r = p/float(Y_train[i])
        if r > emax and py > 7.:
            emax = r
        if r < emin and py > 7.:
            emin = r
#        print(pp,py, r)
    return emin , emax;


def process_model(model, batch_size=32, epochs=500):
    keras_callbacks = [
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
                # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
                EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
                ]
    history = model.fit(X_train, Y_train, batch_size=batch_size,epochs=epochs,
                        shuffle=True, verbose=0, validation_data=(X_test, Y_test), callbacks=keras_callbacks)
                
    train_score = model.evaluate(X_train, Y_train, verbose=0)
    valid_score = model.evaluate(X_test, Y_test, verbose=0)
    
bs=64
rmin=8
rmax=100
for x4 in range(rmin,rmax):
    em = etree_model(x4, dropout=0.2)
    process_model(em, batch_size=64, epochs=5000)
    print(err(em, 50), bs, x4)




Using TensorFlow backend.
W1003 16:07:39.895639  9176 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1003 16:07:39.945010  9176 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1003 16:07:39.950012  9176 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W1003 16:07:39.977015  9176 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

(array([[0.5580431]], dtype=float32), array([[1.4022533]], dtype=float32)) 64 8


C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.55937713]], dtype=float32), array([[1.3976105]], dtype=float32)) 64 9


C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


...

(array([[0.88412964]], dtype=float32), array([[1.291165]], dtype=float32)) 64 70


C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


(array([[0.828774]], dtype=float32), array([[1.6700989]], dtype=float32)) 64 71


C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


KeyboardInterrupt: 